In [1]:
from sympy import symbols, Eq, Function, dsolve, simplify
from sympy.solvers.ode.systems import dsolve_system
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint

In [2]:
# Определение переменных и функций
t = symbols('t')
x = Function('x')(t)
y = Function('y')(t)
a, b, c, d = symbols('a b c d')

# Создание системы дифференциальных уравнений
eq1 = Eq(x.diff(t), a*x + b*y)
eq2 = Eq(y.diff(t), c*x + d*y)

# Решение системы дифференциальных уравнений
solution = dsolve([eq1, eq2])

# Упрощение решения
simplified_solution = [simplify(sol.rhs) for sol in solution]

In [ ]:
# Установка значений параметров и констант
a_val, b_val, c_val, d_val = 1, 2, -1, -2
C1_val, C2_val = 1, 1

# Замена параметров и констант в упрощенных решениях
x_t_simplified = simplified_solution[0].subs({a: a_val, b: b_val, c: c_val, d: d_val, 'C1': C1_val, 'C2': C2_val})
y_t_simplified = simplified_solution[1].subs({a: a_val, b: b_val, c: c_val, d: d_val, 'C1': C1_val, 'C2': C2_val})

# Генерация данных
t_values = np.linspace(0, 10, 100)
x_values = [-2 - np.exp(-val) for val in t_values]
y_values = [1 + np.exp(-val) for val in t_values]

# Построение графиков
plt.figure(figsize=(12, 6))

# График x(t) и y(t)
plt.subplot(1, 2, 1)
plt.plot(t_values, x_values, label='x(t)')
plt.plot(t_values, y_values, label='y(t)')
plt.xlabel('Time (t)')
plt.ylabel('Values')
plt.title('x(t) and y(t)')
plt.legend()

# График y(x)
plt.subplot(1, 2, 2)
plt.scatter(x_values, y_values, c=t_values, cmap='viridis')
plt.xlabel('x')
plt.ylabel('y')
plt.title('y(x) with time color gradient')
plt.colorbar(label='Time (t)')

plt.tight_layout()
plt.show()

In [2]:
# Определение символов
t = symbols('t')
x = Function('x')(t)
y = Function('y')(t)

# Создание системы дифференциальных уравнений
a, b, c, d = symbols('a b c d')
eq1 = Eq(x.diff(t), a*x + b*y)
eq2 = Eq(y.diff(t), c*x + d*y)

# Решение системы ДУ
solutions = dsolve_system([eq1, eq2])
x_solution = solutions[0][0].rhs  # Решение для x(t)
y_solution = solutions[1][0].rhs  # Решение для y(t)

IndexError: list index out of range

In [4]:
# Установка значений параметров и констант
a_val, b_val, c_val, d_val = 1, 2, -1, -2
C1_val, C2_val = 1, 1

# Замена параметров и констант в решениях
x_t = solution[0].rhs.subs({a: a_val, b: b_val, c: c_val, d: d_val, 'C1': C1_val, 'C2': C2_val})
y_t = solution[1].rhs.subs({a: a_val, b: b_val, c: c_val, d: d_val, 'C1': C1_val, 'C2': C2_val})

# Генерация данных
t_values = np.linspace(0, 10, 100)
x_values = [x_t.subs(t, val).evalf() for val in t_values]
y_values = [y_t.subs(t, val).evalf() for val in t_values]

# Построение графиков
plt.figure(figsize=(12, 6))

# График x(t) и y(t)
plt.subplot(1, 2, 1)
plt.plot(t_values, x_values, label='x(t)')
plt.plot(t_values, y_values, label='y(t)')
plt.xlabel('Time (t)')
plt.ylabel('Values')
plt.title('x(t) and y(t)')
plt.legend()

# График y(x)
plt.subplot(1, 2, 2)
plt.scatter(x_values, y_values, c=t_values, cmap='viridis')
plt.xlabel('x')
plt.ylabel('y')
plt.title('y(x) with time color gradient')
plt.colorbar(label='Time (t)')

plt.tight_layout()
plt.show()

AttributeError: 'list' object has no attribute 'rhs'

In [ ]:
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 64),  # Увеличение размера слоя
            nn.ReLU(),         # Изменение функции активации
            nn.Dropout(0.1),   # Добавление Dropout
            nn.Linear(64, 64), # Дополнительный слой
            nn.ReLU(),         # Изменение функции активации
            nn.Linear(64, 2)   # Выходной слой
        )

    def forward(self, t, y):
        return self.net(y)

# Инициализация модели и оптимизатора
func = ODEFunc()
optimizer = optim.Adam(func.parameters(), lr=0.01)

# Подготовка данных
x_train = torch.tensor(x_values, dtype=torch.float32)
y_train = torch.tensor(y_values, dtype=torch.float32)

# Определение количества эпох и начальных условий
num_epochs = 1000  # Примерное количество эпох

# Определение начальных условий
initial_condition_x = x_values[0]
initial_condition_y = y_values[0]

# Определение y0 для NeuralODE
y0 = torch.tensor([initial_condition_x, initial_condition_y], dtype=torch.float32)  # Начальные условия
t_values = torch.linspace(0, 10, 100)  # Временные точки

# Функция потерь
def loss_function(predicted, true):
    return torch.mean((predicted - true)**2)

# Объединение x_values и y_values в один тензор
combined_values = torch.tensor(list(zip(x_values, y_values)), dtype=torch.float32)

# Обучение
for epoch in range(num_epochs):
    optimizer.zero_grad()
    predicted = odeint(func, y0, t_values)
    loss = loss_function(predicted, combined_values)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}: loss = {loss.item()}')

In [ ]:
# Получение весов нейросети
weights = []
for param in func.parameters():
    weights.append(param.detach().numpy())

# Вывод весов
print(weights)

In [ ]:
# Предсказания модели
predicted_values = odeint(func, y0, t_values).detach().numpy()

# Разделение предсказаний на x и y
predicted_x = predicted_values[:, 0]
predicted_y = predicted_values[:, 1]

# Визуализация
plt.figure(figsize=(12, 6))

# Исходные данные
plt.subplot(1, 2, 1)
plt.scatter(x_values, y_values, c='blue', label='Original Data')

# Предсказанные данные
plt.subplot(1, 2, 2)
plt.scatter(predicted_x, predicted_y, c='red', label='Predicted Data')

plt.legend()
plt.show()

In [ ]:
# Добавление шума к исходным данным
noise_level = 0.1
noisy_x_values = x_values + np.random.normal(0, noise_level, len(x_values))
noisy_y_values = y_values + np.random.normal(0, noise_level, len(y_values))

# Подготовка шумных данных для тестирования
noisy_data = torch.tensor(list(zip(noisy_x_values, noisy_y_values)), dtype=torch.float32)

# Предсказание на шумных данных
predicted_noisy = odeint(func, y0, t_values).detach().numpy()

In [ ]:
# Разделение предсказанных данных на x и y
predicted_noisy_x = predicted_noisy[:, 0]
predicted_noisy_y = predicted_noisy[:, 1]

In [ ]:
# Визуализация
plt.figure(figsize=(12, 6))

# Исходные данные
plt.subplot(1, 2, 1)
plt.scatter(x_values, y_values, c='blue', label='Original Data')

# Предсказанные данные на шумных данных
plt.subplot(1, 2, 2)
plt.scatter(predicted_noisy_x, predicted_noisy_y, c='red', label='Predicted on Noisy Data')
plt.scatter(noisy_x_values, noisy_y_values, c='green', alpha=0.5, label='Noisy Data')

plt.legend()
plt.show()

**Решение Системы Дифференциальных Уравнений:**

Мы решили систему линейных дифференциальных уравнений аналитически и получили общие решения в зависимости от времени $t$.

Генерация и визуализация данных:
Сгенерированные данные $x(t)$ и $y(t)$ визуализированы на графиках, что дает представление о поведении системы во времени и о траектории $y(x)$.

Обучение нейросетевой модели (NeuralODE):
Мы использовали нейронную сеть для аппроксимации динамики системы. Обучение модели показало снижение функции потерь, что говорит о том, что нейросеть успешно уловила зависимости между переменными.

Анализ весов модели:
Веса нейросети после обучения представляют аппроксимацию коэффициентов исходной системы. Однако веса, которые мы вывели, не напрямую соотносятся с коэффициентами $a,b,c,d,$ поскольку они представляют параметры слоев нейросети, а не коэффициенты системы ДУ.

Визуализация результатов:
Визуализации показывают сравнение между исходными данными и предсказаниями модели. Это подтверждает, что модель хорошо аппроксимировала данные без шума.

Работа с шумными данными:
После добавления шума к данным и повторного предсказания, визуализация демонстрирует, что модель все еще хорошо аппроксимирует данные, несмотря на наличие шума, хотя точность предсказаний, вероятно, уменьшилась.

In [ ]:
from scipy.optimize import least_squares

initial_condition_x = x_values[0]
initial_condition_y = y_values[0]
y0_tensor = torch.tensor([initial_condition_x, initial_condition_y], dtype=torch.float32)
t_values_tensor = torch.linspace(0, 10, 100)

def neural_net_derivatives(t_tensor, y0_tensor, func):
    # t_tensor уже должен быть тензором PyTorch, нет необходимости его преобразовывать
    with torch.no_grad():
        predicted = odeint(func, y0_tensor, t_tensor)
    # Возвращаем dxdt и dydt, преобразуя предсказанные значения в numpy массивы для дальнейших вычислений
    return predicted[:, 0].numpy(), predicted[:, 1].numpy()

# Функция residuals для использования в методе наименьших квадратов
def residuals(params, t_values, x_values, y_values):
    a, b, c, d = params
    # Убедимся, что x_values и y_values - массивы NumPy
    x_values = np.array(x_values, dtype=float)
    y_values = np.array(y_values, dtype=float)

    # Преобразование numpy массивов обратно в тензоры для передачи в функцию neural_net_derivatives
    t_tensor = torch.from_numpy(t_values).float()
    # Получаем предсказания производных от модели
    dxdt_neural, dydt_neural = neural_net_derivatives(t_tensor, y0_tensor, func)

    # Вычисляем предполагаемые производные на основе параметров модели
    dxdt_predicted = a * x_values + b * y_values
    dydt_predicted = c * x_values + d * y_values

    # Разница между предполагаемыми и предсказанными производными
    residuals_x = dxdt_predicted - dxdt_neural
    residuals_y = dydt_predicted - dydt_neural

    # Возвращаем совокупные остатки как одномерный массив
    return np.concatenate((residuals_x, residuals_y))

# Вызов метода наименьших квадратов для определения коэффициентов
res = least_squares(
    residuals,
    x0=[1, 1, 1, 1],
    args=(t_values_tensor.numpy(), x_values, y_values)
)

print("Найденные коэффициенты:", res.x)

- a = 0.98882262
- b = -0.02187188
- c = 0.00563487
- d = 1.01083844